In [1]:
import luigi
from luigi.util import requires, inherits
from luigi import Task, run, LocalTarget
import wget
import tarfile
import os
import gzip
import pandas as pd
import shutil
import io

In [4]:
class DownloadTask(Task):
    
    url = luigi.Parameter()
    NameDownload = luigi.Parameter()
    dir = luigi.Parameter()
    NameSave = luigi.Parameter()
    
    def run(self):
        fileurl = self.url + self.NameDownload
        wget.download(fileurl, self.output().path)
        
    def output(self):
        return luigi.LocalTarget(self.dir + self.NameSave)
        
@requires(DownloadTask)
class UnzipTar(Task):
    
    def run(self):
        in_file_path = self.input().path
        with tarfile.open(in_file_path, 'r') as tar:
            tar.extractall(self.dir)
    
    def output(self):
        if os.path.exists(self.dir + self.NameSave):
            result = []
            with tarfile.open(self.input().path, 'r') as tar:
                for i in tar.getnames():
                    result.append(luigi.LocalTarget(self.dir + i))
            return result
  
@requires(UnzipTar)
class UnzipGZ(Task):
    
    def run(self):
        # in_file_path = str(self.input())
        # dir_path = in_file_path[:in_file_path.rfind('/')+1]
        branch_count = 1
        
        for i in self.input():
            i = str(i.path)
            dir_name = str(branch_count) + 'branch'
            os.mkdir(self.dir + dir_name)
            with gzip.open(i, 'rb') as f_in:
                archive_name = self.dir + dir_name + '/' + i[i.rfind('/')+1:-2]
                with open(archive_name, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            branch_count += 1
    

# if __name__ == '__main__':
#     run()

In [7]:
params_dict = {'url': 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=',
               'NameDownload': 'GSE68849&format=file',
               'dir': 'Root/',
               'NameSave': 'RootTAR.tar'}
luigi.build([UnzipTar(**params_dict)], workers=1, local_scheduler=True)

DEBUG: Checking if UnzipTar(url=https://www.ncbi.nlm.nih.gov/geo/download/?acc=, NameDownload=GSE68849&format=file, dir=Root/, NameSave=RootTAR.tar) is complete
Traceback (most recent call last):
  File "c:\Users\smart\AppData\Local\Programs\Python\Python39\lib\site-packages\luigi\worker.py", line 441, in check_complete
    is_complete = check_complete_cached(task, completion_cache)
  File "c:\Users\smart\AppData\Local\Programs\Python\Python39\lib\site-packages\luigi\worker.py", line 426, in check_complete_cached
    is_complete = task.complete()
  File "c:\Users\smart\AppData\Local\Programs\Python\Python39\lib\site-packages\luigi\task.py", line 588, in complete
    outputs = flatten(self.output())
  File "C:\Users\smart\AppData\Local\Temp\ipykernel_66392\3275208203.py", line 25, in output
    with tarfile.open(self.input().path, 'r') as tar:
  File "c:\Users\smart\AppData\Local\Programs\Python\Python39\lib\tarfile.py", line 1611, in open
    return func(name, "r", fileobj, **kwargs)
 

False

In [119]:
dfs = {}
with gzip.open('Root/1 branch/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt.gz', 'rb') as f_in:
    with open('Root/1 branch/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        

In [10]:
str = 'Root/1 branch/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt.gz'
str[:-3]

'Root/1 branch/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt'

In [122]:
dfs = {}
with open('Root/1 branch/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt') as f:
    write_key = None
    fio = io.StringIO()
    for l in f.readlines():
        if l.startswith('['):
            if write_key:
                fio.seek(0)
                header = None if write_key == 'Heading' else 'infer'
                dfs[write_key] = pd.read_csv(fio, sep='\t', header=header)
            fio = io.StringIO()
            write_key = l.strip('[]\n')
            continue
        if write_key:
            fio.write(l)
    fio.seek(0)
    dfs[write_key] = pd.read_csv(fio, sep='\t')

In [123]:
for i in dfs:
    dfs[i].to_csv('Root/1 branch/'+i+'.csv', index=False)

In [ ]:
pd.read_csv('Root/1 branch/Probes.csv').head()

In [ ]:
@requires(DownloadTask)
class UnzipTar(Task):
    
    def run(self):
        in_file_path = str(self.input())
        dir_path = in_file_path[:in_file_path.rfind('/')+1]
        branch_count = 1
        
        with tarfile.open(in_file_path, 'r') as tar:
            for i in tar.getnames():
                dir_name = str(branch_count) + 'branch'
                os.mkdir(dir_path + dir_name)
                tar.extract(i, dir_path + dir_name)
                archive_name = dir_path + dir_name + '/' + i
                with gzip.open(archive_name, 'rb') as f_in:
                    with open(archive_name[:-2], 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                branch_count += 1
    
    def output(self):
        pass

In [30]:
from pathlib import Path

In [77]:
class DownloadTask2(Task):
    
    url = luigi.Parameter()
    NameDownload = luigi.Parameter()
    dir = luigi.Parameter()
    NameSave = luigi.Parameter()
    
    def run(self):
        fileurl = self.url + self.NameDownload
        wget.download(fileurl, self.output().path)
        
    def output(self):
        return luigi.LocalTarget(self.dir + self.NameSave)
        
@requires(DownloadTask2)
class UnzipTar2(Task):
    
    def run(self):
        # in_file_path = self.input().path
        with tarfile.open(self.input().path, 'r') as tar:
            tar.extractall(self.dir)
    
    # def output(self):
    #     return luigi.LocalTarget(self.dir)
    def output(self):
        if os.path.exists(self.dir + self.NameSave):
            result = []
            with tarfile.open(self.input().path, 'r') as tar:
                for i in tar.getnames():
                    result.append(luigi.LocalTarget(self.dir + i))
            return result
  
class UnzipGZ2(Task):  
    
    file_name = luigi.Parameter()
    dir = luigi.Parameter()
    dir_num = luigi.IntParameter()
    
    def run(self):
        file_dir = self.dir + self.file_name
        with gzip.open(file_dir, 'rb') as f_in:
            dir_name = 'branch' + str(self.dir_num) + '/'
            os.mkdir(self.dir + dir_name)
            archive_name = self.dir + dir_name + self.file_name[:-3]
            with open(archive_name, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                
    def output(self):
        result = self.dir + 'branch' + str(self.dir_num) + '/' + self.file_name[:-3]
        return luigi.LocalTarget(result)
        
  
@inherits(UnzipTar2)
class UnzipTXT(Task):
    
    def requires(self):
        UnzipTar2(self.url, self.NameDownload, self.dir, self.NameSave)
        task = []
        j = 0
        for i in os.listdir(self.dir):
            i = str(i)
            if i[-3:] == '.gz':
                params = {'file_name': i, 'dir': self.dir, 'dir_num': j}
                task.append(UnzipGZ2(**params))
                j += 1
        return task
    
    def run(self):
        # in_file_path = str(self.input())
        # dir_path = in_file_path[:in_file_path.rfind('/')+1]
        for i in self.input():
            dfs = {}
            with open(i.path) as f:
                write_key = None
                fio = io.StringIO()
                for l in f.readlines():
                    if l.startswith('['):
                        if write_key:
                            fio.seek(0)
                            header = None if write_key == 'Heading' else 'infer'
                            dfs[write_key] = pd.read_csv(fio, sep='\t', header=header)
                        fio = io.StringIO()
                        write_key = l.strip('[]\n')
                        continue
                    if write_key:
                        fio.write(l)
                fio.seek(0)
                dfs[write_key] = pd.read_csv(fio, sep='\t')
            for table in dfs:
                i_str = str(i)
                table_dir = i_str[:i_str.rfind('/')+1]
                dfs[table].to_csv(table_dir + table + '.csv', index=False)
        # branch_count = 1
        
        # for i in self.input():
        #     i = str(i.path)
        #     dir_name = str(branch_count) + 'branch'
        #     os.mkdir(self.dir + dir_name)
        #     with gzip.open(i, 'rb') as f_in:
        #         archive_name = self.dir + dir_name + '/' + i[i.rfind('/')+1:-2]
        #         with open(archive_name, 'wb') as f_out:
        #             shutil.copyfileobj(f_in, f_out)
        #     branch_count += 1

In [78]:
params_dict = {'url': 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=',
               'NameDownload': 'GSE68849&format=file',
               'dir': 'Root2/',
               'NameSave': 'Root.tar'}

luigi.build([UnzipTXT(**params_dict)], workers=1, local_scheduler=True)

DEBUG: Checking if UnzipAllGZ2(url=https://www.ncbi.nlm.nih.gov/geo/download/?acc=, NameDownload=GSE68849&format=file, dir=Root2/, NameSave=Root.tar) is complete
c:\Users\smart\AppData\Local\Programs\Python\Python39\lib\site-packages\luigi\worker.py:426: UserWarning: Task UnzipAllGZ2(url=https://www.ncbi.nlm.nih.gov/geo/download/?acc=, NameDownload=GSE68849&format=file, dir=Root2/, NameSave=Root.tar) without outputs has no custom complete() method
  is_complete = task.complete()
DEBUG: Checking if UnzipGZ2(file_name=GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt.gz, dir=Root2/, dir_num=0) is complete
DEBUG: Checking if UnzipGZ2(file_name=GPL10558_HumanHT-12_V4_0_R2_15002873_B.txt.gz, dir=Root2/, dir_num=1) is complete
INFO: Informed scheduler that task   UnzipAllGZ2_GSE68849_format__Root_tar_Root2__2a042d3a21   has status   PENDING
INFO: Informed scheduler that task   UnzipGZ2_Root2__1_GPL10558_HumanHT_80c87595f8   has status   DONE
INFO: Informed scheduler that task   UnzipGZ2_Root2__0_GP

True

In [22]:
params_dict_gz = {'file_dir': 'Root2/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt.gz',
                  'dir': 'Root2/',
                  'dir_num': 1}

luigi.build([UnzipGZ2(**params_dict_gz)], workers=1, local_scheduler=True)

DEBUG: Checking if UnzipGZ2(file_dir=Root2/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt.gz, dir=Root2/, dir_num=1) is complete
INFO: Informed scheduler that task   UnzipGZ2_Root2__1_Root2_GPL10558_H_797c01fef6   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 66392] Worker Worker(salt=1122116391, workers=1, host=LAPTOP-UU6JICQO, username=smart, pid=66392) running   UnzipGZ2(file_dir=Root2/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt.gz, dir=Root2/, dir_num=1)
INFO: [pid 66392] Worker Worker(salt=1122116391, workers=1, host=LAPTOP-UU6JICQO, username=smart, pid=66392) done      UnzipGZ2(file_dir=Root2/GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt.gz, dir=Root2/, dir_num=1)
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   UnzipGZ2_Root2__1_Root2_GPL10558_H_797c01fef6   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: The

True

In [60]:
a = ('Root2/')
for i in os.listdir(a):
            i = str(i)
            if i[-3:] == '.gz':
                print(i)

GPL10558_HumanHT-12_V4_0_R1_15002873_B.txt.gz
GPL10558_HumanHT-12_V4_0_R2_15002873_B.txt.gz
